In [1]:
# Import our dependencies
import pandas as pd
import os
from pathlib import Path
import csv
import matplotlib.pyplot as plt
import numpy as np
from flask import jsonify
from flask import Flask, render_template, jsonify
from flask_sqlalchemy import SQLAlchemy

In [2]:
# Dependencies and boilerplate
import sqlalchemy
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import MetaData
from sqlalchemy import create_engine,inspect, MetaData

In [3]:
# Create engine using the database file
engine = create_engine("sqlite:///Instructions/DataSets/belly_button_biodiversity.sqlite",echo=True)
# Declare a Base using `automap_base()
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Create a MetaData instance
metadata = MetaData()

# reflect db schema to MetaData
metadata.reflect(bind=engine)

#name the tables
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_Metadata = Base.classes.samples_metadata

2018-03-18 21:28:13,259 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-18 21:28:13,263 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,267 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-18 21:28:13,269 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,272 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-03-18 21:28:13,275 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,280 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("otu")
2018-03-18 21:28:13,282 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,285 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'otu' AND type = 'table'
2018-03-18 21:28:13,287 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,290 INFO sqlalchemy.engine.base.Engi

2018-03-18 21:28:13,492 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,494 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("samples_metadata")
2018-03-18 21:28:13,495 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,497 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("samples_metadata")
2018-03-18 21:28:13,499 INFO sqlalchemy.engine.base.Engine ()
2018-03-18 21:28:13,501 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'samples_metadata' AND type = 'table'
2018-03-18 21:28:13,502 INFO sqlalchemy.engine.base.Engine ()


In [4]:
inspector = inspect(engine)
inspector.get_table_names()

2018-03-18 21:28:13,769 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-03-18 21:28:13,771 INFO sqlalchemy.engine.base.Engine ()


['otu', 'samples', 'samples_metadata']

In [5]:
conn = engine.connect()
print(conn)

In [32]:
def get_columns():
    #lets grab the column names
    column_names=[]
    columns = metadata.tables['samples'].columns.keys()
    column_names = columns[1:len(columns)]
    return (column_names)

test_column_name = get_columns()
print(test_column_name)



['BB_940', 'BB_941', 'BB_943', 'BB_944', 'BB_945', 'BB_946', 'BB_947', 'BB_948', 'BB_949', 'BB_950', 'BB_952', 'BB_953', 'BB_954', 'BB_955', 'BB_956', 'BB_958', 'BB_959', 'BB_960', 'BB_961', 'BB_962', 'BB_963', 'BB_964', 'BB_966', 'BB_967', 'BB_968', 'BB_969', 'BB_970', 'BB_971', 'BB_972', 'BB_973', 'BB_974', 'BB_975', 'BB_978', 'BB_1233', 'BB_1234', 'BB_1235', 'BB_1236', 'BB_1237', 'BB_1238', 'BB_1242', 'BB_1243', 'BB_1246', 'BB_1253', 'BB_1254', 'BB_1258', 'BB_1259', 'BB_1260', 'BB_1264', 'BB_1265', 'BB_1273', 'BB_1275', 'BB_1276', 'BB_1277', 'BB_1278', 'BB_1279', 'BB_1280', 'BB_1281', 'BB_1282', 'BB_1283', 'BB_1284', 'BB_1285', 'BB_1286', 'BB_1287', 'BB_1288', 'BB_1289', 'BB_1290', 'BB_1291', 'BB_1292', 'BB_1293', 'BB_1294', 'BB_1295', 'BB_1296', 'BB_1297', 'BB_1298', 'BB_1308', 'BB_1309', 'BB_1310', 'BB_1374', 'BB_1415', 'BB_1439', 'BB_1441', 'BB_1443', 'BB_1486', 'BB_1487', 'BB_1489', 'BB_1490', 'BB_1491', 'BB_1494', 'BB_1495', 'BB_1497', 'BB_1499', 'BB_1500', 'BB_1501', 'BB_1502'

In [7]:
#conn.execute("select lowest_taxonomic_unit_found from OTU Limit 5 ").fetchall()

In [8]:
def get_otu_desc():
    s=conn.execute("select lowest_taxonomic_unit_found from OTU order by otu_id").fetchall()
    names = [row[0] for row in s]
    return (names)

test_desc = get_otu_desc()
#print(test_desc)

2018-03-18 21:28:16,123 INFO sqlalchemy.engine.base.Engine select lowest_taxonomic_unit_found from OTU order by otu_id
2018-03-18 21:28:16,125 INFO sqlalchemy.engine.base.Engine ()


In [9]:
def get_metadata(sample_number):
    query_param = sample_number[3:]
    
    #print(query_param)
    s = conn.execute("select AGE,BBTYPE, ETHNICITY, GENDER , LOCATION from samples_metadata where sampleid=?",query_param).fetchall()
    print("output of query is",s)
     
    
       
    bb_metadata={"Age" : s[0][0],
                 "BBTYPE":s[0][1],
                 "ETHNICITY":s[0][2],
                 "GENDER":s[0][3],
                 "LOCATION":s[0][4],
                 "SAMPLEID":query_param
                }
    return(bb_metadata)

In [10]:
test_metadata = get_metadata('BB_948')
print(test_metadata)

2018-03-18 21:28:17,501 INFO sqlalchemy.engine.base.Engine select AGE,BBTYPE, ETHNICITY, GENDER , LOCATION from samples_metadata where sampleid=?
2018-03-18 21:28:17,504 INFO sqlalchemy.engine.base.Engine ('948',)
output of query is [(20, 'I', 'Caucasian', 'M', 'ChapelHill/NC')]
{'Age': 20, 'BBTYPE': 'I', 'ETHNICITY': 'Caucasian', 'GENDER': 'M', 'LOCATION': 'ChapelHill/NC', 'SAMPLEID': '948'}


In [11]:
def get_wfreq(otu_id):
    input_bb = otu_id
    search_query=input_bb.split("_")
    query_param=search_query[1]
    wfreq_data = conn.execute("select WFREQ from samples_metadata where sampleid=?",query_param).fetchone() 
    return(wfreq_data[0])





In [12]:
test_metadata = get_wfreq('BB_940')
print(test_metadata)

2018-03-18 21:28:18,779 INFO sqlalchemy.engine.base.Engine select WFREQ from samples_metadata where sampleid=?
2018-03-18 21:28:18,781 INFO sqlalchemy.engine.base.Engine ('940',)
2


In [23]:
def get_otu_id(sample_number):
    query_param = sample_number
    otu_id=[]
    distribution=[]
    pie_dict =[]

    #print(query_param)
    s = conn.execute("select  otu_id , BB_940 from samples order by BB_940 desc").fetchall()
    #print("output of query is",s)
    print("outputin function",s[0:15])
    for i in range(len(s[0:15])):
        print(s[i][0])
        print(s[i][1])
        otu_id.append(s[i][0])
        distribution.append(s[i][1])
    pie_dict.append(otu_id)
    pie_dict.append(distribution)
    print(pie_dict)
    return pie_dict
    
      
 

In [24]:
test_dict = get_otu_id('BB_940')

2018-03-18 21:39:41,388 INFO sqlalchemy.engine.base.Engine select  otu_id , BB_940 from samples order by BB_940 desc
2018-03-18 21:39:41,390 INFO sqlalchemy.engine.base.Engine ()
outputin function [(1167, 163), (2859, 126), (482, 113), (2264, 78), (41, 71), (1189, 51), (352, 50), (189, 47), (1977, 40), (2318, 40), (3450, 37), (874, 36), (1959, 30), (2191, 28), (1950, 25)]
1167
163
2859
126
482
113
2264
78
41
71
1189
51
352
50
189
47
1977
40
2318
40
3450
37
874
36
1959
30
2191
28
1950
25
[[1167, 2859, 482, 2264, 41, 1189, 352, 189, 1977, 2318, 3450, 874, 1959, 2191, 1950], [163, 126, 113, 78, 71, 51, 50, 47, 40, 40, 37, 36, 30, 28, 25]]


In [25]:
#Create a PIE chart that uses data from your routes /samples/<sample> and /otu to display the top 10 samples.
#Use the Sample Value as the values for the PIE chart -
#Use the OTU ID as the labels for the pie chart
#Use the OTU Description as the hovertext for the chart

otu_id = get_otu_id('BB_940')[0]
print(otu_id)
descrip = get_otu_id('BB_940')[1]
print(descrip)

2018-03-18 21:39:50,091 INFO sqlalchemy.engine.base.Engine select  otu_id , BB_940 from samples order by BB_940 desc
2018-03-18 21:39:50,094 INFO sqlalchemy.engine.base.Engine ()
outputin function [(1167, 163), (2859, 126), (482, 113), (2264, 78), (41, 71), (1189, 51), (352, 50), (189, 47), (1977, 40), (2318, 40), (3450, 37), (874, 36), (1959, 30), (2191, 28), (1950, 25)]
1167
163
2859
126
482
113
2264
78
41
71
1189
51
352
50
189
47
1977
40
2318
40
3450
37
874
36
1959
30
2191
28
1950
25
[[1167, 2859, 482, 2264, 41, 1189, 352, 189, 1977, 2318, 3450, 874, 1959, 2191, 1950], [163, 126, 113, 78, 71, 51, 50, 47, 40, 40, 37, 36, 30, 28, 25]]
[1167, 2859, 482, 2264, 41, 1189, 352, 189, 1977, 2318, 3450, 874, 1959, 2191, 1950]
2018-03-18 21:39:50,101 INFO sqlalchemy.engine.base.Engine select  otu_id , BB_940 from samples order by BB_940 desc
2018-03-18 21:39:50,115 INFO sqlalchemy.engine.base.Engine ()
outputin function [(1167, 163), (2859, 126), (482, 113), (2264, 78), (41, 71), (1189, 51), (